In [1]:
import tensorflow as tf
import numpy as np


from tensorflow.python.layers.core import Dense

In [2]:
with open('letters_source.txt', 'r', encoding='utf-8') as f:
    source_data = f.read()

with open('letters_target.txt', 'r', encoding='utf-8') as f:
    target_data = f.read()

In [3]:
source_data.split('\n')[:10]

['bsaqq',
 'npy',
 'lbwuj',
 'bqv',
 'kial',
 'tddam',
 'edxpjpg',
 'nspv',
 'huloz',
 'kmclq']

In [4]:
target_data.split('\n')[:10]

['abqqs',
 'npy',
 'bjluw',
 'bqv',
 'aikl',
 'addmt',
 'degjppx',
 'npsv',
 'hlouz',
 'cklmq']

In [5]:
def extract_character_vocab(data):
    '''
    构造映射表
    '''
    special_words = ['<PAD>', '<UNK>', '<GO>',  '<EOS>']

    set_words = list(set([character for line in data.split('\n') for character in line]))
    print('set_words',set_words)
    # 这里要把四个特殊字符添加进词典
    int_to_vocab = {idx: word for idx, word in enumerate(special_words + set_words)}
    vocab_to_int = {word: idx for idx, word in int_to_vocab.items()}
    
    #将字典的键和值进行颠倒

    return int_to_vocab, vocab_to_int

In [6]:
set_words = list(set([character for line in source_data.split('\n') for character in line]))
#set()对字符进行了去重复，只留下非重复字符
# print('set_words',set_words)[0:10]
# set_words ['b', 's', 'a', 'q', 'q', 'n', 'p', 'y', 'l', 'b', 'w', 'u', 'j', 'b', 'q', 'v', 'k', 'i', 'a', 'l', 't', 'd', 'd', 'a', 'm', 'e', 'd', 'x', 'p', 'j', 'p', 'g', 'n', 's', 'p', 'v', 'h', 'u', 'l', '

In [7]:
#元组里面放了两个字典
extract_character_vocab(source_data)

set_words ['r', 'u', 'k', 'd', 't', 's', 'b', 'i', 'w', 'q', 'g', 'y', 'j', 'o', 'p', 'c', 'v', 'x', 'm', 'z', 'a', 'f', 'l', 'n', 'e', 'h']


({0: '<PAD>',
  1: '<UNK>',
  2: '<GO>',
  3: '<EOS>',
  4: 'r',
  5: 'u',
  6: 'k',
  7: 'd',
  8: 't',
  9: 's',
  10: 'b',
  11: 'i',
  12: 'w',
  13: 'q',
  14: 'g',
  15: 'y',
  16: 'j',
  17: 'o',
  18: 'p',
  19: 'c',
  20: 'v',
  21: 'x',
  22: 'm',
  23: 'z',
  24: 'a',
  25: 'f',
  26: 'l',
  27: 'n',
  28: 'e',
  29: 'h'},
 {'<PAD>': 0,
  '<UNK>': 1,
  '<GO>': 2,
  '<EOS>': 3,
  'r': 4,
  'u': 5,
  'k': 6,
  'd': 7,
  't': 8,
  's': 9,
  'b': 10,
  'i': 11,
  'w': 12,
  'q': 13,
  'g': 14,
  'y': 15,
  'j': 16,
  'o': 17,
  'p': 18,
  'c': 19,
  'v': 20,
  'x': 21,
  'm': 22,
  'z': 23,
  'a': 24,
  'f': 25,
  'l': 26,
  'n': 27,
  'e': 28,
  'h': 29})

In [8]:
source_int_to_letter, source_letter_to_int = extract_character_vocab(source_data)
target_int_to_letter, target_letter_to_int = extract_character_vocab(target_data)

set_words ['r', 'u', 'k', 'd', 't', 's', 'b', 'i', 'w', 'q', 'g', 'y', 'j', 'o', 'p', 'c', 'v', 'x', 'm', 'z', 'a', 'f', 'l', 'n', 'e', 'h']
set_words ['r', 'u', 'k', 'd', 't', 's', 'b', 'i', 'w', 'q', 'g', 'y', 'j', 'o', 'p', 'c', 'v', 'x', 'm', 'z', 'a', 'f', 'l', 'n', 'e', 'h']


In [9]:
[[source_letter_to_int.get(letter, source_letter_to_int['<UNK>']) for letter in line]
 for line in source_data.split('\n')][0:10]

[[10, 9, 24, 13, 13],
 [27, 18, 15],
 [26, 10, 12, 5, 16],
 [10, 13, 20],
 [6, 11, 24, 26],
 [8, 7, 7, 24, 22],
 [28, 7, 21, 18, 16, 18, 14],
 [27, 9, 18, 20],
 [29, 5, 26, 17, 23],
 [6, 22, 19, 26, 13]]

In [10]:
# 对字母进行转换dict.get(key, default=None),返回指定键的值，default -- 如果指定键的值不存在时，返回该默认值值。
source_int = [[source_letter_to_int.get(letter, source_letter_to_int['<UNK>']) for letter in line] 
              for line in source_data.split('\n')]

#target端首先在最后加上eos作为结束标志
target_int = [[target_letter_to_int.get(letter, target_letter_to_int['<UNK>']) for letter in line] 
              + [target_letter_to_int['<EOS>']] 
              for line in target_data.split('\n')]

In [11]:
source_int[:10]

[[10, 9, 24, 13, 13],
 [27, 18, 15],
 [26, 10, 12, 5, 16],
 [10, 13, 20],
 [6, 11, 24, 26],
 [8, 7, 7, 24, 22],
 [28, 7, 21, 18, 16, 18, 14],
 [27, 9, 18, 20],
 [29, 5, 26, 17, 23],
 [6, 22, 19, 26, 13]]

In [12]:
target_int[:10]

[[24, 10, 13, 13, 9, 3],
 [27, 18, 15, 3],
 [10, 16, 26, 5, 12, 3],
 [10, 13, 20, 3],
 [24, 11, 6, 26, 3],
 [24, 7, 7, 22, 8, 3],
 [7, 28, 14, 16, 18, 18, 21, 3],
 [27, 18, 9, 20, 3],
 [29, 26, 17, 5, 23, 3],
 [19, 6, 26, 22, 13, 3]]

In [13]:
def get_inputs():
    '''
    模型输入tensor
    '''
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    # 定义target序列最大长度（之后target_sequence_length和source_sequence_length会作为feed_dict的参数）
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, targets, learning_rate, target_sequence_length, max_target_sequence_length, source_sequence_length

In [14]:
def get_encoder_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):

    '''
    构造Encoder层
    
    参数说明：
    - input_data: 输入tensor
    - rnn_size: rnn隐层结点数量
    - num_layers: 堆叠的rnn cell数量
    - source_sequence_length: 源数据的序列长度
    - source_vocab_size: 源数据的词典大小
    - encoding_embedding_size: embedding的大小
    '''
    # Encoder embedding
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return lstm_cell

    cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, encoder_embed_input, 
                                                      sequence_length=source_sequence_length, dtype=tf.float32)
    
    return encoder_output, encoder_state

In [15]:
def process_decoder_input(data, vocab_to_int, batch_size):
    '''
    补充<GO>，并移除最后一个字符；##此处仅仅是将最后一个元素去掉，并且在最前面添加元素go
    '''
    # cut掉最后一个字符
    ending = tf.strided_slice(data, [0, 0], [batch_size, -1], [1, 1])
    
    #创建dims=[batch_size, 1],value=vocab_to_int['<GO>']的矩阵，后面拼接ending
    decoder_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return decoder_input

In [16]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, encoder_state, decoder_input):
    '''
    构造Decoder层
    
    参数：
    - target_letter_to_int: target数据的映射表
    - decoding_embedding_size: embed向量大小
    - num_layers: 堆叠的RNN单元数量
    - rnn_size: RNN单元的隐层结点数量
    - target_sequence_length: target数据序列长度
    - max_target_sequence_length: target数据序列最大长度
    - encoder_state: encoder端编码的状态向量
    - decoder_input: decoder端输入
    '''
    # 1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)

    # 2. 构造Decoder中的RNN单元
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return decoder_cell
    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Output全连接层
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Training decoder
    with tf.variable_scope("decode"):
        # 得到help对象
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        # 构造decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        training_decoder_output, _,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
    # 5. Predicting decoder
    # 与training共享参数
    with tf.variable_scope("decode", reuse=True):
        # 创建一个常量tensor并复制为batch_size的大小
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32), [batch_size], 
                               name='start_tokens')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                start_tokens,
                                                                target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                        predicting_helper,
                                                        encoder_state,
                                                        output_layer)
        predicting_decoder_output, _,_ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)
    
    return training_decoder_output, predicting_decoder_output

In [26]:
def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  encoder_embedding_size, decoder_embedding_size, 
                  rnn_size, num_layers):
    
    # 获取encoder的状态输出
    _, encoder_state = get_encoder_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # 预处理后的decoder输入
    decoder_input = process_decoder_input(targets, target_letter_to_int, batch_size)
    print('targets',targets,'\n****\n')
    
    print('process_decoder_input',process_decoder_input,'\n****\n')
    
    # 将状态向量与输入传递给decoder
    training_decoder_output, predicting_decoder_output = decoding_layer(target_letter_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       encoder_state, 
                                                                       decoder_input) 
    
    return training_decoder_output, predicting_decoder_output

In [31]:
# 超参数
# Number of Epochs
epochs = 80
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001


# In[13]:

# 构造graph
train_graph = tf.Graph()

with train_graph.as_default():
    
    # 获得模型输入    
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_inputs()
    
    #调用模型
    training_decoder_output, predicting_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_letter_to_int),
                                                                      len(target_letter_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')
    
    process_decoder_input_example=process_decoder_input(targets, target_letter_to_int, batch_size)

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

targets Tensor("targets:0", shape=(?, ?), dtype=int32) 
****

process_decoder_input <function process_decoder_input at 0x11a88cd90> 
****



In [28]:
def pad_sentence_batch(sentence_batch, pad_int):
    '''
    对batch中的序列进行补全，保证batch中的每行都有相同的sequence_length
    
    参数：
    - sentence batch
    - pad_int: <PAD>对应索引号
    '''
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [29]:
def get_batches(targets, sources, batch_size, source_pad_int, target_pad_int):
    '''
    定义生成器，用来获取batch
    '''
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        # 补全序列
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))
        
        # 记录每条记录的长度
        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))
        
        source_lengths = []
        for source in sources_batch:
            source_lengths.append(len(source))
        
        yield pad_targets_batch, pad_sources_batch, targets_lengths, source_lengths

In [35]:
# 将数据集分割为train和validation
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# 留出一个batch进行验证
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]
(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 50 # 每隔50轮输出loss

checkpoint = "./trained_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(train_target, train_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>'])):
            
            # print('targets_batch, sources_batch\n',targets_batch,'\n****\n', sources_batch,'\n****\n')
            '''
            {0: '<PAD>',
              1: '<UNK>',
              2: '<GO>',
              3: '<EOS>'
            
            targets_batch, 
             [[25 27  5 ...  0  0  0]
             [16  6  3 ...  0  0  0]
             [14 18 13 ... 12  3  0]
             ...
             [14 29 22 ... 12  3  0]
             [19 14  4 ...  3  0  0]
             [24 11 11 ...  8 20  3]] 
             
             将targets_batch最后一个元素去掉，在最前面添加go的id即可获得decoder-input
             decoder-input 
             [[ 2 25 27 ...  0  0  0]
             [ 2 16  6 ...  0  0  0]
             [ 2 14 18 ...  8 12  3]
             ...
             [ 2 14 29 ... 12 12  3]
             [ 2 19 14 ... 12  3  0]
             [ 2 24 11 ...  9  8 20]]
            ****
            sources_batch
             [[ 5 25 27  0  0  0  0]
             [ 6 16  0  0  0  0  0]
             [13 12  8 18 14 13  0]
             [12  8 26  0  0  0  0]
             [29 20  0  0  0  0  0]
             [27 20 27  0  0  0  0]
            '''
            
            #eos，pad之后开始运行seq2seq_model进行decoder端的去掉eos添加go
            #调试看是什么
            print('decoder-input',sess.run(process_decoder_input_example,{input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths}))
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            if batch_i % display_step == 0:
                
                # 计算validation loss
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))

    
    
    # 保存模型
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

decoder-input [[ 2 25 27 ...  0  0  0]
 [ 2 16  6 ...  0  0  0]
 [ 2 14 18 ...  8 12  3]
 ...
 [ 2 14 29 ... 12 12  3]
 [ 2 19 14 ... 12  3  0]
 [ 2 24 11 ...  9  8 20]]


Epoch   1/80 Batch    0/77 - Training Loss:  3.402  - Validation loss:  3.397
decoder-input [[ 2 24  3 ...  0  0  0]
 [ 2  6  6 ...  0  0  0]
 [ 2 10 10 ...  0  0  0]
 ...
 [ 2 10 10 ...  5 20  3]
 [ 2 24 10 ... 14  3  0]
 [ 2 29  8 ...  0  0  0]]
decoder-input [[ 2 25  4 ...  3  0  0]
 [ 2 27 17 ... 12  3  0]
 [ 2 11 16 ...  3  0  0]
 ...
 [ 2 28 29 ...  4  3  0]
 [ 2  6 27 ...  3  0  0]
 [ 2 25  6 ...  0  0  0]]
decoder-input [[ 2 22 18 ... 20 15  3]
 [ 2 16 26 ... 18  3  0]
 [ 2 19  7 ...  3  0  0]
 ...
 [ 2  7 16 ...  0  0  0]
 [ 2 28 21 ...  0  0  0]
 [ 2 24 22 ...  3  0  0]]
decoder-input [[ 2 26 27 ...  0  0  0]
 [ 2 28 25 ...  0  0  0]
 [ 2 29 11 ...  9 21  3]
 ...
 [ 2 20  3 ...  0  0  0]
 [ 2  7  9 ...  0  0  0]
 [ 2 17 20 ...  0  0  0]]
decoder-input [[ 2  7  9 ...  0  0  0]
 [ 2 16  6 ...  4 20  3]
 [ 2 29 16 ...  9 12  3]
 ...
 [ 2 28  4 ...  3  0  0]
 [ 2 24 28 ...  5 20 20]
 [ 2  7 26 ...  3  0  0]]
decoder-input [[ 2  6 27 ...  0  0  0]
 [ 2 14 16 ... 12 21 15]
 [ 2 10 

decoder-input [[ 2 11 12 ...  0  0  0]
 [ 2 10 10 ...  3  0  0]
 [ 2 10 27 ... 17  9 21]
 ...
 [ 2  7 11 ...  9  8  3]
 [ 2 19 19 ...  8 15 23]
 [ 2 24  7 ...  0  0  0]]
decoder-input [[ 2 14  6 ...  5 15  3]
 [ 2 14  6 ... 20  3  0]
 [ 2 24 10 ...  0  0  0]
 ...
 [ 2 24  6 ... 12 15  3]
 [ 2 10 19 ... 20 15  3]
 [ 2 14 11 ... 18  4 21]]
decoder-input [[ 2  7 14 ...  0  0  0]
 [ 2 29 27 ...  0  0  0]
 [ 2 28 17 ...  0  0  0]
 ...
 [ 2 22  4 ...  0  0  0]
 [ 2 26  3 ...  0  0  0]
 [ 2 29  3 ...  0  0  0]]
decoder-input [[ 2 24 28 ... 29 16 15]
 [ 2  7 25 ...  0  0  0]
 [ 2 25 14 ...  3  0  0]
 ...
 [ 2 19 28 ... 12 15  3]
 [ 2 24 19 ... 11 26 26]
 [ 2 21  3 ...  0  0  0]]
decoder-input [[ 2 29 21 ...  0  0  0]
 [ 2 19  3 ...  0  0  0]
 [ 2 10 10 ... 16 26  9]
 ...
 [ 2 24 22 ...  4  3  0]
 [ 2 24 16 ...  0  0  0]
 [ 2 14 11 ... 13 20  3]]
decoder-input [[ 2 11 27 ... 23  3  0]
 [ 2 10 22 ...  9 12  3]
 [ 2 10 19 ... 18  8  3]
 ...
 [ 2 29 15 ...  0  0  0]
 [ 2 29 11 ...  0  0  0]
 [ 2 1

decoder-input [[ 2  6 13 ...  0  0  0]
 [ 2  9 21 ...  0  0  0]
 [ 2 16 16 ...  3  0  0]
 ...
 [ 2  7  3 ...  0  0  0]
 [ 2 24  7 ... 20  3  0]
 [ 2  7  4 ...  0  0  0]]
decoder-input [[ 2 22  3 ...  0  0  0]
 [ 2 16 15 ...  0  0  0]
 [ 2 10 22 ...  0  0  0]
 ...
 [ 2 14 21 ...  0  0  0]
 [ 2 20  3 ...  0  0  0]
 [ 2 25 26 ...  3  0  0]]
decoder-input [[ 2 24 24 ...  5  3  0]
 [ 2 24  8 ... 15 23  3]
 [ 2 24 14 ...  0  0  0]
 ...
 [ 2 24 16 ...  8 21 15]
 [ 2 24 19 ... 12 15 23]
 [ 2 10 19 ...  6  4  3]]
decoder-input [[ 2 19 27 ...  0  0  0]
 [ 2 25 14 ...  3  0  0]
 [ 2 19 19 ... 14 11 17]
 ...
 [ 2 28 25 ...  0  0  0]
 [ 2 10 11 ... 18  5 20]
 [ 2 24 28 ... 15  3  0]]
decoder-input [[ 2  7 11 ...  0  0  0]
 [ 2 24 26 ... 13  4 12]
 [ 2  4  3 ...  0  0  0]
 ...
 [ 2 19 11 ...  0  0  0]
 [ 2  5 12 ...  0  0  0]
 [ 2 13 23 ...  0  0  0]]
decoder-input [[ 2 14 23 ...  0  0  0]
 [ 2 11  6 ... 12 15  3]
 [ 2 28 29 ... 20 20 23]
 ...
 [ 2 10  3 ...  0  0  0]
 [ 2 11  3 ...  0  0  0]
 [ 2 1

decoder-input [[ 2 28 14 ... 17  9 12]
 [ 2 19 29 ...  5  3  0]
 [ 2 24 14 ...  3  0  0]
 ...
 [ 2 10 11 ... 13  3  0]
 [ 2  7 29 ...  3  0  0]
 [ 2 11 26 ... 18 23  3]]
decoder-input [[ 2 10 10 ... 17  3  0]
 [ 2 10 27 ...  5 23  3]
 [ 2  7 14 ... 13 12 21]
 ...
 [ 2 24 11 ... 13  4 23]
 [ 2  7  7 ...  3  0  0]
 [ 2 19 14 ...  0  0  0]]
decoder-input [[ 2 22 18 ...  0  0  0]
 [ 2  7 29 ...  8 20  3]
 [ 2 15  3 ...  0  0  0]
 ...
 [ 2 27  3 ...  0  0  0]
 [ 2 10 13 ... 20 21 23]
 [ 2 29 12 ...  0  0  0]]
decoder-input [[ 2  6  3 ...  0  0  0]
 [ 2 28 11 ...  0  0  0]
 [ 2 24 16 ...  0  0  0]
 ...
 [ 2 23  3 ...  0  0  0]
 [ 2 25 26 ... 15 15  3]
 [ 2  4 15 ...  0  0  0]]
decoder-input [[ 2  7  3 ...  0  0  0]
 [ 2 11 27 ...  5 12 23]
 [ 2 28 25 ... 22  3  0]
 ...
 [ 2 19  3 ...  0  0  0]
 [ 2  4 21 ...  0  0  0]
 [ 2 14  6 ...  0  0  0]]
decoder-input [[ 2 16 26 ...  5 12  3]
 [ 2 10 28 ...  0  0  0]
 [ 2  9  5 ...  0  0  0]
 ...
 [ 2 10 16 ...  0  0  0]
 [ 2 10 19 ... 23  3  0]
 [ 2 1

...

decoder-input [[ 2 12  3 ...  0  0  0]
 [ 2 19 29 ...  0  0  0]
 [ 2 10  7 ... 13 20 21]
 ...
 [ 2 28 27 ...  3  0  0]
 [ 2 28 14 ... 20 12  3]
 [ 2 22  3 ...  0  0  0]]
decoder-input [[ 2 19  7 ... 13  5  3]
 [ 2 25 25 ... 27 18  3]
 [ 2 10 11 ...  6 15 23]
 ...
 [ 2  9  3 ...  0  0  0]
 [ 2 24  7 ...  9  5  3]
 [ 2 19 28 ...  3  0  0]]
decoder-input [[ 2 19  7 ... 26 21 15]
 [ 2 10 19 ...  3  0  0]
 [ 2 19  3 ...  0  0  0]
 ...
 [ 2 24 16 ... 21 23  3]
 [ 2  6 26 ... 21  3  0]
 [ 2 25 29 ... 18 13  4]]
decoder-input [[ 2 25 29 ...  0  0  0]
 [ 2 17  5 ...  0  0  0]
 [ 2  7 14 ...  8  8 23]
 ...
 [ 2  6  4 ...  0  0  0]
 [ 2 25 27 ...  3  0  0]
 [ 2 16  6 ...  3  0  0]]
decoder-input [[ 2 10 19 ...  3  0  0]
 [ 2  7 16 ...  0  0  0]
 [ 2 10 19 ... 12 12  3]
 ...
 [ 2 29 16 ...  3  0  0]
 [ 2 19 28 ...  0  0  0]
 [ 2 22 27 ...  9 15  3]]
decoder-input [[ 2 28 14 ... 17 13  8]
 [ 2  7 13 ... 20 12  3]
 [ 2  7 29 ... 18  8  3]
 ...
 [ 2 29 29 ...  9 20 23]
 [ 2 27  9 ... 15  3  0]
 [ 2 2

decoder-input [[ 2 26 27 ...  0  0  0]
 [ 2 28 25 ...  0  0  0]
 [ 2 29 11 ...  9 21  3]
 ...
 [ 2 20  3 ...  0  0  0]
 [ 2  7  9 ...  0  0  0]
 [ 2 17 20 ...  0  0  0]]
decoder-input [[ 2  7  9 ...  0  0  0]
 [ 2 16  6 ...  4 20  3]
 [ 2 29 16 ...  9 12  3]
 ...
 [ 2 28  4 ...  3  0  0]
 [ 2 24 28 ...  5 20 20]
 [ 2  7 26 ...  3  0  0]]
decoder-input [[ 2  6 27 ...  0  0  0]
 [ 2 14 16 ... 12 21 15]
 [ 2 10 19 ...  3  0  0]
 ...
 [ 2 27  3 ...  0  0  0]
 [ 2  7 14 ...  9  3  0]
 [ 2 25 13 ...  0  0  0]]
decoder-input [[ 2 11 11 ...  3  0  0]
 [ 2 10  3 ...  0  0  0]
 [ 2 19 14 ... 18  9  9]
 ...
 [ 2 29  4 ...  0  0  0]
 [ 2 28 11 ...  0  0  0]
 [ 2 24 19 ... 29 11 15]]
decoder-input [[ 2 29 16 ...  3  0  0]
 [ 2 11  3 ...  0  0  0]
 [ 2 10  7 ...  4 23  3]
 ...
 [ 2  6 27 ... 20 15  3]
 [ 2 28 29 ...  3  0  0]
 [ 2 28 11 ...  8  3  0]]
decoder-input [[ 2  4  8 ...  0  0  0]
 [ 2 19 18 ...  0  0  0]
 [ 2 10 14 ... 17 15  3]
 ...
 [ 2 27  3 ...  0  0  0]
 [ 2  7 28 ...  3  0  0]
 [ 2 1

decoder-input [[ 2 24 11 ...  0  0  0]
 [ 2 28 16 ...  0  0  0]
 [ 2 16  9 ...  0  0  0]
 ...
 [ 2 13 13 ... 12  3  0]
 [ 2 11  3 ...  0  0  0]
 [ 2 10 28 ...  0  0  0]]
decoder-input [[ 2  7  4 ...  0  0  0]
 [ 2 29 11 ... 15  3  0]
 [ 2 24 26 ... 18  5 15]
 ...
 [ 2 14  4 ... 15 23  3]
 [ 2 24 10 ... 12  3  0]
 [ 2 14  3 ...  0  0  0]]
decoder-input [[ 2 24  7 ... 18 15  3]
 [ 2 10 13 ...  0  0  0]
 [ 2 25 27 ...  8  3  0]
 ...
 [ 2 25 22 ... 15  3  0]
 [ 2 28  6 ...  0  0  0]
 [ 2 11 11 ...  3  0  0]]
decoder-input [[ 2 13  8 ...  0  0  0]
 [ 2 29 16 ... 17 20  3]
 [ 2 24  3 ...  0  0  0]
 ...
 [ 2 24 19 ... 27  9 15]
 [ 2  6 17 ...  0  0  0]
 [ 2 24 29 ...  3  0  0]]
decoder-input [[ 2 25 17 ...  0  0  0]
 [ 2 29 23 ...  0  0  0]
 [ 2  5  3 ...  0  0  0]
 ...
 [ 2 26  3 ...  0  0  0]
 [ 2 19 17 ...  0  0  0]
 [ 2  5  3 ...  0  0  0]]
decoder-input [[ 2 14  6 ...  9 15  3]
 [ 2 24 25 ...  3  0  0]
 [ 2 25  3 ...  0  0  0]
 ...
 [ 2 28  3 ...  0  0  0]
 [ 2 12  3 ...  0  0  0]
 [ 2 2

decoder-input [[ 2 14 23 ...  0  0  0]
 [ 2 11  6 ... 12 15  3]
 [ 2 28 29 ... 20 20 23]
 ...
 [ 2 10  3 ...  0  0  0]
 [ 2 11  3 ...  0  0  0]
 [ 2 19 14 ... 13 21  3]]
decoder-input [[ 2 28 25 ...  3  0  0]
 [ 2 29  6 ...  9 21  3]
 [ 2 28 16 ... 18  5  3]
 ...
 [ 2  7  9 ...  0  0  0]
 [ 2 27 18 ...  0  0  0]
 [ 2 27 13 ...  3  0  0]]
decoder-input [[ 2 10  7 ...  3  0  0]
 [ 2  6 27 ... 15  3  0]
 [ 2 14 29 ... 22  4 15]
 ...
 [ 2 25 13 ...  0  0  0]
 [ 2  6 27 ... 12 15  3]
 [ 2  6  6 ... 23  3  0]]
decoder-input [[ 2 24 18 ... 21  3  0]
 [ 2 10 19 ... 16 26 18]
 [ 2 11 27 ...  0  0  0]
 ...
 [ 2 19 14 ...  6 27  3]
 [ 2 16 22 ... 15 23  3]
 [ 2 10 10 ...  4 15 23]]
decoder-input [[ 2 17  4 ...  8 15 23]
 [ 2  6  9 ...  3  0  0]
 [ 2 11 27 ...  0  0  0]
 ...
 [ 2 16 20 ...  0  0  0]
 [ 2 19 22 ...  0  0  0]
 [ 2 24 19 ... 20  3  0]]
decoder-input [[ 2 28 13 ...  0  0  0]
 [ 2 25 11 ...  0  0  0]
 [ 2 28 29 ...  5  3  0]
 ...
 [ 2 19 19 ...  8 21 15]
 [ 2 28 28 ...  8  3  0]
 [ 2 1

KeyboardInterrupt: 

In [38]:
import os
print(os.path.abspath('.'))

/Users/ozintel/Downloads/seq2seq_eg


In [44]:
def source_to_seq(text):
    '''
    对源数据进行转换
    '''
    sequence_length = 7
    return [source_letter_to_int.get(word, source_letter_to_int['<UNK>']) for word in text] + [source_letter_to_int['<PAD>']]*(sequence_length-len(text))

In [49]:
# 输入一个单词
input_word = 'iogmhk'
text = source_to_seq(input_word)

checkpoint = "./trained_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # 加载模型
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(input_word)]*batch_size, 
                                      source_sequence_length: [len(input_word)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('原始输入:', input_word)

print('\nSource')
print('  Word 编号:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word 编号:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./trained_model.ckpt


原始输入: iogmhk

Source
  Word 编号:    [29, 12, 13, 24, 18, 14, 0]
  Input Words: i o g m h k <PAD>

Target
  Word 编号:       [13, 18, 29, 14, 24, 12]
  Response Words: g h i k m o
